In [76]:
import os
import logging 
import requests

from datetime import datetime

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

diretorio_dados = 'dados'
diretorio_dados_processados = os.path.join(diretorio_dados, "processados")
diretorio_imagens = 'imagens'
arquivo_csv = "merged_data.csv"


In [36]:
def reset_logger(logger_name, log_file_path):
    """
    Resets the logger by removing handlers, clearing the log file, 
    and reinitializing the logger.
    
    Parameters:
        logger_name (str): The name of the logger to reset.
        log_file_path (str): Path to the log file to clear and reuse.
    """
    logger = logging.getLogger(logger_name)
    
    # Remove and close all handlers
    for handler in logger.handlers[:]:
        handler.close()
        logger.removeHandler(handler)
    
    # Clear the log file
    with open(log_file_path, "w", encoding="utf-8") as log_file:
        log_file.write("")
    
    # Reinitialize the logger
    logging.basicConfig(
        filename=log_file_path,
        level=logging.INFO,
        format="%(asctime)s - %(levelname)s - %(message)s",
        encoding="utf-8"
    )
    
# Nome do logger e caminho do arquivo de log
logger_name = "root"  # Nome do logger padrão
arquivo_log = "dados_alterados.log"

# Configurar o logger inicialmente
logging.basicConfig(
    filename=arquivo_log,
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    encoding="utf-8"
)

logging.info("Logger inicializado.")

In [34]:
# Finalizar o logger e fechar o arquivo de log
for handler in logging.root.handlers[:]:
    handler.close()
    logging.root.removeHandler(handler)

In [2]:
# Função para buscar dados da API do Banco Central
def fetch_bcb_series(series_code, start_date, end_date=None):
    """
    Busca uma série do Banco Central do Brasil usando a API SGS.
    
    Args:
        series_code (int): Código da série.
        start_date (str): Data de início no formato DD/MM/AAAA.
        end_date (str): Data de fim no formato DD/MM/AAAA (default: hoje).
    
    Returns:
        pd.DataFrame: DataFrame com as datas e valores da série.
    """
    if end_date is None:
        end_date = datetime.today().strftime('%d/%m/%Y')
    
    url = f"https://api.bcb.gov.br/dados/serie/bcdata.sgs.{series_code}/dados"
    params = {"formato": "json", "dataInicial": start_date, "dataFinal": end_date}
    print(url)
    print(params)
    response = requests.get(url, params=params)
    response.raise_for_status()
    
    data = response.json()
    df = pd.DataFrame(data)
    df['data'] = pd.to_datetime(df['data'], format='%d/%m/%Y')
    df['valor'] = pd.to_numeric(df['valor'], errors='coerce')
    return df

In [ ]:
# Lista de séries do BCB e seus códigos
bcb_series = [
    { 
        "Nome": "Selic",
        "Descrição" : "Taxa de juros que representa a taxa média ajustada das operações compromissadas com prazo de um dia útil lastreadas com títulos públicos federais custodiados no Sistema Especial de Liquidação e de Custódia (Selic). Divulgação em % a.d.",
        "Série": 11,
    },
    { 
        "Nome": "Taxa de Juros PF",
        "Descrição" : "Taxa de juros que representa a taxa média ajustada das operações compromissadas com prazo de um dia útil lastreadas com títulos públicos federais custodiados no Sistema Especial de Liquidação e de Custódia (Selic). Divulgação em % a.d.",
        "Série": 11,
    },
    {
        "Nome": "Spread médio PF",
        "Descrição": "Diferença entre a taxa média de juros das novas operações de crédito livre contratadas no período de referência e o custo de captação referencial médio. Não inclui operações referenciadas em taxas regulamentadas, operações vinculadas a recursos do Banco Nacional de Desenvolvimento Econômico e Social (BNDES) ou quaisquer outras lastreadas em recursos compulsórios ou governamentais.",
        "Série": 20809,
    },
    {
        "Nome": "Concessões de crédito PF",
        "Descrição": "Conceito: Valor das novas operações de crédito contratadas no período de referência com taxas de juros livremente pactuadas entre mutuários e instituições financeiras. Não inclui operações referenciadas em taxas regulamentadas, operações vinculadas a recursos do Banco Nacional de Desenvolvimento Econômico e Social (BNDES) ou quaisquer outras lastreadas em recursos compulsórios ou governamentais.",
        "Série": 20662,
    },
    {
        "Nome": "Inadimplência da carteira de crédito PF",
        "Descrição": "Conceito: Percentual da carteira de crédito do Sistema Financeiro Nacional com pelo menos uma parcela com atraso superior a 90 dias. Inclui operações contratadas no segmento de crédito livre e no segmento de crédito direcionado.",
        "Série": 21084,
    },
    {
        "Nome": "Captação líquida diária de depósitos de poupança - SBPE",
        "Descrição": "A captação líquida é representada pela soma das aplicações e dos rendimentos creditados, excluindo-se as retiradas",
        "Série": 240,
    },
    {
        "Nome": "Saldo diário de depósitos de poupança - SBPE e rural",
        "Descrição": "Saldo diário de depósitos de poupança - SBPE e rural",
        "Série": 23,
    },
    {
        "Nome": "Índice de Confiança do Consumidor",
        "Descrição": "O Índice de Confiança do Consumidor (ICC) busca identificar o sentimento dos consumidores relativo às suas condições financeiras, às suas perspectivas futuras e também a percepção que o consumidor tem das condições econômicas do país.",
        "Série": 4393,
    },
    {
        "Nome": "Índice nacional de preços ao consumidor-amplo (IPCA)",
        "Descrição": "O IPCA tem por objetivo medir a inflação de um conjunto de produtos e serviços comercializados no varejo, referentes ao consumo pessoal das famílias. Para garantir uma cobertura de 90% das famílias das áreas urbanas do País, abrange atualmente aquelas com rendimentos de 1 a 40 salários mínimos nas regiões metropolitanas de Belém, Fortaleza, Recife, Salvador, Belo Horizonte, Vitória, Rio de Janeiro, São Paulo, Curitiba e Porto Alegre, além do Distrito Federal e dos municípios de Goiânia, Campo Grande, Rio Branco, São Luís e Aracaju. (Var. % mensal) ",
        "Série": 433,
    },
]

In [20]:
# Data inicial para busca
start_date = "01/01/2000"

# Loop para buscar todas as séries do BCB
for item in bcb_series:
    name = item["Nome"]
    code = item["Série"]
    print(f"Baixando série do BCB: {name} (Código: {code})")
    df = fetch_bcb_series(code, start_date)
    df = df.rename(columns={"valor": name})
    filename = f"BCB {name} - COD{code} - {datetime.today().strftime('%Y%m%d%H%M%S')}.csv"
    filename = os.path.join(diretorio_dados, filename)
    df.to_csv(filename, index=False)
    print(f"Arquivo salvo: {filename}")


Baixando série do BCB: Selic (Código: 11)
https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados
{'formato': 'json', 'dataInicial': '01/01/2000', 'dataFinal': '12/11/2024'}
Arquivo salvo: dados\BCB Selic - COD11 - 20241112144802.csv
Baixando série do BCB: Taxa de Juros PF (Código: 11)
https://api.bcb.gov.br/dados/serie/bcdata.sgs.11/dados
{'formato': 'json', 'dataInicial': '01/01/2000', 'dataFinal': '12/11/2024'}
Arquivo salvo: dados\BCB Taxa de Juros PF - COD11 - 20241112144802.csv
Baixando série do BCB: Spread médio PF (Código: 20809)
https://api.bcb.gov.br/dados/serie/bcdata.sgs.20809/dados
{'formato': 'json', 'dataInicial': '01/01/2000', 'dataFinal': '12/11/2024'}
Arquivo salvo: dados\BCB Spread médio PF - COD20809 - 20241112144803.csv
Baixando série do BCB: Concessões de crédito PF (Código: 20662)
https://api.bcb.gov.br/dados/serie/bcdata.sgs.20662/dados
{'formato': 'json', 'dataInicial': '01/01/2000', 'dataFinal': '12/11/2024'}
Arquivo salvo: dados\BCB Concessões de crédito PF - 

In [22]:
# Função para buscar dados da API do IBGE (SIDRA)
def fetch_ibge_series(table_id, variable_id, period="all"):
    """
    Busca dados do IBGE usando a API SIDRA.
    
    Args:
        table_id (int): Código da tabela no SIDRA.
        variable_id (int): Código da variável na tabela.
        period (str): Período de busca (default: "all").
    
    Returns:
        pd.DataFrame: DataFrame com os dados da série.
    """
    url = f"https://apisidra.ibge.gov.br/values/t/{table_id}/v/{variable_id}/p/{period}/N1/1"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    print(url)
    
    # Converter JSON para DataFrame
    df = pd.DataFrame(data[1:])  # Ignora o cabeçalho
    df['data'] = pd.to_datetime(df['D2C'].str[:4] + '-' + (df['D2C'].str[4:].astype(int) * 3 - 2).astype(str), format='%Y-%m')
    df['valor'] = pd.to_numeric(df['V'], errors='coerce')
    return df[['data', 'valor']]

In [23]:
# Lista de séries do IBGE e suas tabelas/variáveis
ibge_series = {
    "Índice de Desemprego": {
        "table_id": 4094,
        "variable_id": 4098,
        "descrição": "Tabela: 4094: Pessoas de 14 anos ou mais de idade, total, na força de trabalho, ocupadas, desocupadas, fora da força de trabalho, em situação de informalidade e respectivas taxas e níveis, por grupo de idade. Variável: 4098:  Nível da desocupação, na semana de referência, das pessoas de 14 anos ou mais de idade (%)",
        },  # Taxa de desemprego
}

# Loop para buscar todas as séries do IBGE
for name, params in ibge_series.items():
    print(f"Baixando série do IBGE: {name} (Tabela: {params['table_id']}, Variável: {params['variable_id']})")
    df = fetch_ibge_series(params['table_id'], params['variable_id'])
    df = df.rename(columns={"valor": name})
    filename = f"IBGE {name} - T{params['table_id']} - V{params['variable_id']} - {datetime.today().strftime('%Y%m%d%H%M%S')}.csv"
    filename = os.path.join(diretorio_dados, filename)
    df.to_csv(filename, index=False)
    print(f"Arquivo salvo: {filename}")

Baixando série do IBGE: Índice de Desemprego (Tabela: 4094, Variável: 4098)
https://apisidra.ibge.gov.br/values/t/4094/v/4098/p/all/N1/1
Arquivo salvo: dados\IBGE Índice de Desemprego - T4094 - V4098 - 20241112144928.csv


In [79]:
def fetch_pib_data(name):
    """
    Busca os dados do PIB trimestral usando a API do IBGE.
    """
    url = "https://apisidra.ibge.gov.br/values/t/6613/p/all/v/9319/N1/1/c11255/90707"
    response = requests.get(url)
    response.raise_for_status()
    data = response.json()
    print(url)
    
    # Converter JSON para DataFrame
    df = pd.DataFrame(data[1:])  # Ignora o cabeçalho
    df['data'] = pd.to_datetime(df['D1C'].str[:4] + '-' + (df['D1C'].str[4:].astype(int) * 3 - 2).astype(str), format='%Y-%m')    
    df['PIB'] = pd.to_numeric(df['V'], errors='coerce')
    return df[['data', 'PIB']]

In [80]:
descricao = "Tabela: 6613: Valores encadeados a preços de 1995 com ajuste sazonal. Variável: 9319:  Valores encadeados a preços de 1995 com ajuste sazonal (Milhões de Reais) - casas decimais: padrão = 2, máximo = 4. Setor: 90707  PIB a preços de mercado"

# Testar a função
name = 'PIB Desazonalizado Encadeado 1995'
df_pib = fetch_pib_data(name)
filename = f"IBGE {name}  - T6613 - V9319 - {datetime.today().strftime('%Y%m%d%H%M%S')}.csv"
filename = os.path.join(diretorio_dados, filename)

# Exportar para CSV (opcional)
df_pib.to_csv(filename, index=False)

https://apisidra.ibge.gov.br/values/t/6613/p/all/v/9319/N1/1/c11255/90707


In [99]:
# Função para processar arquivos CSV
def plotar_graficos():
    sns.set_theme(style="whitegrid")
    for arquivo in os.listdir(diretorio_dados):
        if arquivo.endswith('.csv'):
            caminho_arquivo = os.path.join(diretorio_dados, arquivo)

            # Leitura do arquivo CSV
            df = pd.read_csv(caminho_arquivo)

            # Verifica se as colunas necessárias existem
            if 'data' not in df.columns or len(df.columns) < 2:
                print(f"Aviso: O arquivo {arquivo} não contém as colunas necessárias.")
                continue

            # Obtém o nome da segunda coluna (que será usada como título e eixo Y)
            coluna_dados = df.columns[1]

            # Converte a coluna 'data' para datetime para garantir formato correto
            df['data'] = pd.to_datetime(df['data'], format='%Y-%m-%d', errors='coerce')

            # Remove valores inválidos
            df = df.dropna(subset=['data', coluna_dados])

            # Nome do arquivo de saída
            nome_imagem = f"{coluna_dados}.png"
            caminho_imagem = os.path.join(diretorio_imagens, nome_imagem)

            # Criação do gráfico
            plt.figure(figsize=(10, 6))
            sns.lineplot(data=df, x='data', y=coluna_dados, marker='o')
            plt.title(f"{coluna_dados}")
            plt.xlabel("Data")
            plt.ylabel(coluna_dados)
            plt.tight_layout()

            # Salvar gráfico
            plt.savefig(caminho_imagem, format='png')
            plt.close()
            print(f"Gráfico salvo: {caminho_imagem}")

In [100]:
plotar_graficos()

Gráfico salvo: imagens\Captação líquida diária de depósitos de poupança - SBPE.png
Gráfico salvo: imagens\Concessões de crédito PF.png
Gráfico salvo: imagens\Inadimplência da carteira de crédito PF.png
Gráfico salvo: imagens\Saldo diário de depósitos de poupança - SBPE e rural.png
Gráfico salvo: imagens\Selic.png
Gráfico salvo: imagens\Spread médio PF.png
Gráfico salvo: imagens\Taxa de Juros PF.png
Gráfico salvo: imagens\Índice de Confiança do Consumidor.png
Gráfico salvo: imagens\Índice nacional de preços ao consumidor-amplo (IPCA).png
Gráfico salvo: imagens\PIB.png
Gráfico salvo: imagens\Índice de Desemprego.png


In [83]:
# Datas dos primeiros dias de cada trimestre
primeiros_dias_trimestre = ["01-01", "04-01", "07-01", "10-01"]

# Função para processar arquivos CSV
def preencher_dados_faltantes():

    # Resetar o logger
    reset_logger(logger_name, arquivo_log)

    logging.info("Logger resetado e iniciado novamente.")

    logging.info("Iniciando o processamento dos arquivos CSV.")

    # Itera sobre os arquivos no diretório
    for arquivo in os.listdir(diretorio_dados):
        if arquivo.endswith(".csv"):
            caminho_arquivo = os.path.join(diretorio_dados, arquivo)            
            logging.info(f"Processando arquivo: {arquivo}")

            try:
                # Leitura do arquivo CSV
                df = pd.read_csv(caminho_arquivo)

                # Verifica se as colunas necessárias existem
                if 'data' not in df.columns:
                    logging.warning(f"O arquivo {arquivo} não contém a coluna 'data'. Ignorado.")
                    continue

                # Converte a coluna 'data' para datetime
                df['data'] = pd.to_datetime(df['data'], format='%Y-%m-%d', errors='coerce')

                # Remove valores inválidos e ordena por data
                df = df.dropna(subset=['data']).sort_values(by='data').reset_index(drop=True)

                # Garante que o índice é datetime
                df.set_index('data', inplace=True)

                # Determina os anos com dados no arquivo
                anos = df.index.year.unique()

                # Processa cada ano
                alteracoes = []
                for ano in anos:
                    for dia in primeiros_dias_trimestre:
                        data_trimestre = pd.Timestamp(f"{ano}-{dia}")

                        if data_trimestre not in df.index:  # Se a data não existe
                            # Busca o primeiro dado do trimestre
                            inicio_trimestre = pd.Timestamp(f"{ano}-{dia}")
                            fim_trimestre = inicio_trimestre + pd.offsets.QuarterEnd(0)
                            filtro = (df.index >= inicio_trimestre) & (df.index <= fim_trimestre)

                            if filtro.any():  # Se houver dados no trimestre
                                dado_origem = df.loc[filtro].iloc[0]
                                df.loc[data_trimestre] = dado_origem
                                alteracoes.append((str(data_trimestre.date()), str(dado_origem.name.date())))

                # Ordena novamente o DataFrame após inserir dados
                df.sort_index(inplace=True)

                # Salva o arquivo CSV atualizado
                df.reset_index(inplace=True)

                caminho_arquivo_processado = os.path.join(diretorio_dados_processados, 'p'+arquivo)
                df.to_csv(caminho_arquivo_processado, index=False)

                # Registra alterações no log
                for alteracao in alteracoes:
                    logging.info(f"{arquivo} - Data alterada: {alteracao[0]} - Origem: {alteracao[1]}")

                logging.info(f"Arquivo processado com sucesso: {arquivo}")

            except Exception as e:
                logging.error(f"Erro ao processar o arquivo {arquivo}: {e}")

    logging.info("Processamento concluído.")

In [84]:
preencher_dados_faltantes()

In [85]:
def merge_csv_files_in_folder(folder_path):
    """
    Escaneia todos os arquivos .csv em uma pasta e faz o merge usando a coluna 'data' como índice.

    Args:
        folder_path (str): Caminho da pasta onde os arquivos CSV estão armazenados.

    Returns:
        pd.DataFrame: DataFrame combinado.
    """
    merged_df = None
    
    # Escanear arquivos na pasta
    for file_name in os.listdir(folder_path):
        if file_name.endswith('.csv'):
            file_path = os.path.join(folder_path, file_name)
            print(f"Lendo arquivo: {file_name}")
            # Carregar o arquivo CSV
            df = pd.read_csv(file_path, index_col='data')
            
            # Merge com o DataFrame principal
            if merged_df is None:
                merged_df = df
            else:
                merged_df = merged_df.merge(df, how='inner', left_index=True, right_index=True)

            print(f'Quantidade de linhas no arquivo final: {len(merged_df)}')
    
    return merged_df

In [86]:
# Chama a função para combinar os arquivos e descartar linhas com dados faltando
merged_df = merge_csv_files_in_folder(diretorio_dados_processados).dropna()

# Exportar o DataFrame combinado para um arquivo CSV (opcional)
merged_df.to_csv(arquivo_csv)

Lendo arquivo: pBCB Captação líquida diária de depósitos de poupança - SBPE - COD240 - 20241112144804.csv
Quantidade de linhas no arquivo final: 6276
Lendo arquivo: pBCB Concessões de crédito PF - COD20662 - 20241112144803.csv
Quantidade de linhas no arquivo final: 124
Lendo arquivo: pBCB Inadimplência da carteira de crédito PF - COD21084 - 20241112144803.csv
Quantidade de linhas no arquivo final: 124
Lendo arquivo: pBCB Saldo diário de depósitos de poupança - SBPE e rural - COD23 - 20241112144810.csv
Quantidade de linhas no arquivo final: 124
Lendo arquivo: pBCB Selic - COD11 - 20241112144802.csv
Quantidade de linhas no arquivo final: 124
Lendo arquivo: pBCB Spread médio PF - COD20809 - 20241112144803.csv
Quantidade de linhas no arquivo final: 124
Lendo arquivo: pBCB Taxa de Juros PF - COD11 - 20241112144802.csv
Quantidade de linhas no arquivo final: 124
Lendo arquivo: pBCB Índice de Confiança do Consumidor - COD4393 - 20241112144811.csv
Quantidade de linhas no arquivo final: 124
Lend

In [104]:
# Configurar estilo do seaborn
sns.set_theme(style="white")

# Criar a pasta de imagens se não existir
os.makedirs(diretorio_imagens, exist_ok=True)

# Ler o arquivo CSV
df = pd.read_csv(arquivo_csv)

# Verificar se a coluna 'data' existe e remover antes de calcular a correlação
if 'data' in df.columns:
    df.drop(columns=['data'], inplace=True)

# Calcular a matriz de correlação
correlacao = df.corr()

# Criar máscara para mostrar apenas o quadrante inferior
mask = np.triu(np.ones_like(correlacao, dtype=bool))

# Criar o gráfico de correlação
plt.figure(figsize=(10, 8))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(
    correlacao, 
    mask=mask, 
    cmap=cmap, 
    annot=True, 
    fmt=".2f", 
    square=True, 
    cbar_kws={"shrink": 0.8}, 
    linewidths=0.5
)

# Título opcional
plt.title("Matriz de Correlação (Quadrante Inferior)", fontsize=14)

# Salvar a imagem em SVG
caminho_svg = os.path.join(diretorio_imagens, "matriz_correlacao.png")
plt.savefig(caminho_svg, format="png",bbox_inches="tight")
plt.close()

print(f"Matriz de correlação salva em: {caminho_svg}")

Matriz de correlação salva em: imagens\matriz_correlacao.png


In [105]:
# Configurar estilo do seaborn
sns.set_theme(style="whitegrid")

# Ler o arquivo CSV
df = pd.read_csv(arquivo_csv)

# Verificar se a coluna 'data' existe e removê-la, pois não é necessária para a correlação
if 'data' in df.columns:
    df.drop(columns=['data'], inplace=True)

# Garantir que as colunas SELIC e PIB existem
if 'Selic' not in df.columns or 'PIB' not in df.columns:
    raise ValueError("As colunas 'SELIC' e 'PIB' devem estar presentes no arquivo.")

# Selecionar colunas para análise (todas exceto SELIC e PIB)
variaveis = [col for col in df.columns if col not in ['SELIC', 'PIB']]

# Função para calcular correlação defasada
def calcular_correlacao_defasada(df, referencia, variaveis, max_defasagem):
    """
    Calcula a correlação defasada entre uma variável de referência e outras variáveis.
    
    Parameters:
        df (pd.DataFrame): DataFrame com os dados.
        referencia (str): Nome da variável de referência.
        variaveis (list): Lista de variáveis para correlacionar com a referência.
        max_defasagem (int): Máxima defasagem (positiva e negativa) para calcular.
    
    Returns:
        pd.DataFrame: Matriz de correlação defasada.
    """
    correlacoes = pd.DataFrame(index=range(-max_defasagem, max_defasagem + 1), columns=variaveis)

    for lag in range(-max_defasagem, max_defasagem + 1):
        for var in variaveis:
            # Aplicar defasagem à variável de referência
            if lag < 0:
                correlacoes.loc[lag, var] = df[referencia].shift(-lag).corr(df[var])
            else:
                correlacoes.loc[lag, var] = df[referencia].shift(lag).corr(df[var])

    return correlacoes.astype(float)

# Máxima defasagem
max_defasagem = 4

# Calcular correlação defasada para PIB e SELIC
correlacao_pib = calcular_correlacao_defasada(df, 'PIB', variaveis, max_defasagem)
correlacao_selic = calcular_correlacao_defasada(df, 'Selic', variaveis, max_defasagem)

# Função para plotar e salvar a matriz de correlação defasada
def plotar_matriz_correlacao(correlacoes, titulo, nome_arquivo):
    plt.figure(figsize=(12, 8))

    # Substituir valores ausentes por 0 ou outro valor
    correlacoes = correlacoes.fillna(0).T  # Transpor para inverter os eixos

    sns.heatmap(
        correlacoes,
        cmap="coolwarm",
        annot=True,
        fmt=".2f",
        cbar_kws={"shrink": 0.8},
        linewidths=0.5
    )
    plt.title(titulo, fontsize=16)
    plt.xlabel("Defasagem")
    plt.ylabel("Variáveis")
    plt.tight_layout()

    # Salvar a matriz como SVG
    caminho_svg = os.path.join(diretorio_imagens, nome_arquivo)
    plt.savefig(caminho_svg, format="png", bbox_inches="tight")
    plt.close()
    print(f"Gráfico salvo em: {caminho_svg}")

# Plotar e salvar as matrizes de correlação defasada
plotar_matriz_correlacao(correlacao_pib, "Matriz de Correlação Defasada (PIB)", "correlacao_defasada_pib.png")
plotar_matriz_correlacao(correlacao_selic, "Matriz de Correlação Defasada (SELIC)", "correlacao_defasada_selic.png")


Gráfico salvo em: imagens\correlacao_defasada_pib.png
Gráfico salvo em: imagens\correlacao_defasada_selic.png
